# Tirage aléatoire de stems

- à partir du fichier Stems.yaml
- vers le fichiers randomStems.yaml

In [22]:
# -*- coding: utf8 -*-
#########################IMPORTS############################################
from os.path import expanduser
import codecs, optparse
import re
import random as rd
import sys,os,time
import string
import yaml, YamlDuplicates
import ParFuMor as PFM
from ParFuMor import *
import pickle

In [2]:
from string import translate, maketrans

In [6]:
home = expanduser("~")
# repertoire=home+"/ownCloud/Cours/Bordeaux/L1-UE4-Morphologie/00-Kanonik/"+annee
repertoire=home+"/Github/Kalaba/Kalaba2/Kanonik/"
repertoire=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/22-K5/"

with open(repertoire+"Stems.yaml", 'r') as stream:
    nullStems=yaml.load(stream,Loader=yaml.Loader)
    
noStems=True
prevStems=nullStems.copy()
setRadicals=set()

In [34]:
nomsPropres=["Clemencia","Elphaba","Hoderi","Kaleb","Mahira","Violette","Nabil","Katisha","Nicole"]
nomsPropresP=["klemensja","elfaba","oderi","kaleb","maira","vjolet","nabil","katiSa","nikol"]

In [39]:
with open(repertoire+"Phonology.yaml", 'r') as stream:
    phonology=yaml.load(stream,Loader=yaml.Loader)

In [13]:
obstrusivesSourdes=u"ptkfTsS"
obstrusivesVoisees=u"bdgvDzZ"

voisement=maketrans(obstrusivesSourdes,obstrusivesVoisees)
devoisement=maketrans(obstrusivesVoisees,obstrusivesSourdes)


consonnes=phonology["consonnes"]
voyelles=phonology["voyelles"]

In [16]:
consonnes

'ptkbdgmnNfsTSvzDZrljw'

In [23]:
from numpy.random import choice

consonnesFreq=[10,10,10,10,10,10,8,8,6,7,7,7,7,5,5,5,5,8,8,3,3]
consonnesTot=sum(consonnesFreq)
consonnesProb=[float(k)/consonnesTot for k in consonnesFreq]


In [24]:
def randomCVCVC():
    result=""
    result+=choice(list(consonnes), p=consonnesProb) #from numpy
    result+=rd.choice(voyelles)                      #from random
    result+=choice(list(consonnes), p=consonnesProb)
    result+=rd.choice(voyelles)
    result+=choice(list(consonnes), p=consonnesProb)
    if result not in setRadicals:
        return result
    else:
        return randomCVCVC()

In [25]:
if noStems:
    for cat in nullStems:
        prevStems[cat]={}
        for lex in nullStems[cat]:
            rdRadical=randomCVCVC()
            prevStems[cat][rdRadical]=nullStems[cat][lex]
            

In [26]:
def getLeaves(stemStructure):
    formes={}
    for element in stemStructure:
        if type(stemStructure[element])==list:
            formesCle=", ".join(stemStructure[element])
            formes[formesCle]=element
        else:
            formes.update(getLeaves(stemStructure[element]))
    return formes

In [27]:
prevLexique=getLeaves(prevStems)
nextLexique=getLeaves(prevStems)

In [28]:
def replace_keys(old_dict, key_dict):
    new_dict = { }
    for key in old_dict.keys():
        new_key = key_dict.get(key, key)
        if isinstance(old_dict[key], dict):
            new_dict[new_key] = replace_keys(old_dict[key], key_dict)
        else:
            new_dict[new_key] = old_dict[key]
    return new_dict


In [29]:
def devoiserFinale(radical):
    if radical[-1] in obstrusivesVoisees:
        return radical.translate(devoisement)
    else:
        return None

def voiserVCV(radical):
    result=list(radical)
    cibles=[]
    for p,l in enumerate(radical[1:-1]):
        if l in obstrusivesSourdes and radical[p] in voyelles and radical[p+2] in voyelles:
            cibles.append(p+1)
    if cibles:
        cible=rd.choice(cibles)
        result[cible]=result[cible].translate(voisement)
        return "".join(result)
    else:
        return None

def metathese(radical):
    result=list(radical)
    cibles=[]
    for p,l in enumerate(radical):
        if l in consonnes:
            cibles.append(p)
    if len(cibles)>1 and rd.randint(1,3)==1:
        cible1=rd.choice(cibles[:-1])
        cible2=cibles[cibles.index(cible1)+1]
        result[cible1]=radical[cible2]
        result[cible2]=radical[cible1]
        result="".join(result)
        if result!=radical:
            return result
        else:
            return None
    else:
        return None

    
def apophonie(radical):
    result=list(radical)
    cibles=[]
    for p,l in enumerate(radical):
        if l in "iae":
            cibles.append(p)
    if cibles:
        cible=rd.choice(cibles)
        result[cible]=phonology["apophonies"][result[cible]]
        return "".join(result)
    else:
        return None

def changerLongueur(radical):
    result=list(radical)
    if len(radical)==5:
        if radical[0] in consonnes:
            if radical[-1] in consonnes:
                if rd.randint(1,5)==1:
                    result.append(rd.choice(voyelles))
                    return "".join(result)
                else:
                    return None
            else:
                if rd.randint(1,5)==1:
                    result.append(rd.choice(consonnes))
                    return "".join(result)
                else:
                    return None
        else:
            if rd.randint(1,3)==1:
                result.insert(0,rd.choice(consonnes))
                return "".join(result)
            else:
                return None
    elif len(radical)>5:
        if rd.randint(1,3)==1:
            if radical[0] in consonnes:
                result=result[:-1]
            else:
                result=result[1:]
            return "".join(result)
        else:
            return None
    else:
        if rd.randint(1,3)==1:
            if radical[0] in voyelles:
                result.insert(0,rd.choice(consonnes))
            elif radical[-1] in voyelles:
                result.append(rd.choice(consonnes))
            else:
                result.append(rd.choice(voyelles))
            return "".join(result)
        else:
            return None
        
            

In [30]:
operations=[devoiserFinale,devoiserFinale,voiserVCV,voiserVCV,changerLongueur,changerLongueur,apophonie,metathese]

In [35]:
def randomRadical(radical):
    result=""
    for letter in radical:
        if letter in voyelles:
            result+=rd.choice(voyelles)
        else:
            result+=choice(list(consonnes), p=consonnesProb)
    print "random",result
    return result

def modifierRadical(radical,dist=[0,1,2,2,3,3,3,4,5]):
    nOperations=rd.choice(dist)
    while nOperations>0:
        operation=rd.choice(operations)
        print nOperations,operation
        result=operation(radical)
        if result:
            radical=result
            nOperations-=1
    print "modif",radical
    return radical

In [36]:
modifierRadical(randomRadical(u"kaleb"),dist=[0,0,0,1,1,2])

random nebed
modif nebed


'nebed'

In [41]:
racineChanges={}
for mot in prevLexique:
    if mot in nextLexique:
        if prevLexique[mot] in nomsPropresP:
            racineChanges[nextLexique[mot]]=prevLexique[mot]
        else:
            racineChanges[nextLexique[mot]]=modifierRadical(randomRadical(prevLexique[mot]),dist=[0,0,0,1,1,2])
    else:
        print "***"
        print mot,"n'est pas dans le nouveau lexique"
        print "***"

random pokigo
1 <function changerLongueur at 0x7fce095446e0>
1 <function apophonie at 0x7fce09544578>
modif pokago
random tapuD
modif tapuD
random ponaf
1 <function devoiserFinale at 0x7fce095445f0>
1 <function metathese at 0x7fce09544488>
modif pofan
random goseTj
modif goseTj
random pemap
modif pemap
random vafup
modif vafup
random faveb
modif faveb
random SeDiT
modif SeDiT
random befej
modif befej
random budup
1 <function apophonie at 0x7fce09544578>
1 <function apophonie at 0x7fce09544578>
1 <function devoiserFinale at 0x7fce095445f0>
1 <function apophonie at 0x7fce09544578>
1 <function devoiserFinale at 0x7fce095445f0>
1 <function changerLongueur at 0x7fce095446e0>
1 <function apophonie at 0x7fce09544578>
1 <function voiserVCV at 0x7fce09544668>
1 <function voiserVCV at 0x7fce09544668>
1 <function metathese at 0x7fce09544488>
1 <function voiserVCV at 0x7fce09544668>
1 <function voiserVCV at 0x7fce09544668>
1 <function voiserVCV at 0x7fce09544668>
1 <function changerLongueur at 0x7

random NoNap
2 <function voiserVCV at 0x7fce09544668>
2 <function voiserVCV at 0x7fce09544668>
2 <function devoiserFinale at 0x7fce095445f0>
2 <function devoiserFinale at 0x7fce095445f0>
2 <function changerLongueur at 0x7fce095446e0>
1 <function voiserVCV at 0x7fce09544668>
modif NoNabi
random jiSap
1 <function metathese at 0x7fce09544488>
modif Sijap
random Zefal
modif Zefal
random Tipat
1 <function apophonie at 0x7fce09544578>
modif Tapat
random jotif
modif jotif
random Sokit
1 <function apophonie at 0x7fce09544578>
modif Sokat
random epubin
2 <function devoiserFinale at 0x7fce095445f0>
2 <function apophonie at 0x7fce09544578>
1 <function voiserVCV at 0x7fce09544668>
modif obubin
random bakiN
2 <function voiserVCV at 0x7fce09544668>
1 <function apophonie at 0x7fce09544578>
modif bagaN
random muvaz
modif muvaz
random TotuS
modif TotuS
random liSoZ
modif liSoZ
random lubob
modif lubob
random lesen
1 <function voiserVCV at 0x7fce09544668>
modif lezen
random mekun
1 <function devoiserFin

In [42]:
modifierRadical("Tonif")

5 <function voiserVCV at 0x7fce09544668>
5 <function metathese at 0x7fce09544488>
4 <function apophonie at 0x7fce09544578>
3 <function devoiserFinale at 0x7fce095445f0>
3 <function devoiserFinale at 0x7fce095445f0>
3 <function devoiserFinale at 0x7fce095445f0>
3 <function voiserVCV at 0x7fce09544668>
2 <function devoiserFinale at 0x7fce095445f0>
2 <function voiserVCV at 0x7fce09544668>
2 <function changerLongueur at 0x7fce095446e0>
2 <function apophonie at 0x7fce09544578>
1 <function apophonie at 0x7fce09544578>
1 <function changerLongueur at 0x7fce095446e0>
1 <function changerLongueur at 0x7fce095446e0>
1 <function apophonie at 0x7fce09544578>
1 <function voiserVCV at 0x7fce09544668>
1 <function changerLongueur at 0x7fce095446e0>
1 <function changerLongueur at 0x7fce095446e0>
1 <function apophonie at 0x7fce09544578>
1 <function changerLongueur at 0x7fce095446e0>
modif Tovuno


'Tovuno'

In [43]:
racineChanges

{'Dagam': 'bulad',
 'Dalib': 'bagaN',
 'DeTap': 'vafof',
 'Degem': 'dovaT',
 'DiZat': 'gazoZ',
 'DoDal': 'tebose',
 'Dodor': 'nuZor',
 'DofaT': 'netok',
 'Dubav': 'deTom',
 'DuguN': 'todido',
 'Duwif': 'liteS',
 'Named': 'ZiDuN',
 'NiSes': 'soduSe',
 'Niben': 'tonug',
 'Nikos': 'sudef',
 'Nipet': 'koSak',
 'Nipik': 'diwir',
 'NuDuf': 'Tebeb',
 'NuNef': 'biToS',
 'Nubif': 'gesok',
 'Safed': 'pemap',
 'SakaT': 'tapuD',
 'SatoT': 'gamut',
 'Sebep': 'ZuSuT',
 'Sejok': 'Sipuzu',
 'Sezev': 'dovomu',
 'SiZuf': 'loSib',
 'Sibur': 'tupes',
 'SodaT': 'tiduT',
 'Solod': 'modoj',
 'Soniv': 'tosaS',
 'Sosak': 'buzil',
 'Spuk': 'gpeD',
 'SukoD': 'pikop',
 'SukuD': 'goriN',
 'Sures': 'masuz',
 'Sutib': 'NipiS',
 'TaDaz': 'SeDiT',
 'TaDod': 'bolab',
 'Tejiv': 'vafup',
 'Tigos': 'Sulowi',
 'Tilap': 'nekado',
 'TizuS': 'Zipan',
 'Tobaz': 'resew',
 'Tofed': 'bakud',
 'Tofow': 'movus',
 'Tonud': 'tarar',
 'Tugat': 'Zagum',
 'Tugif': 'teDaf',
 'Tukat': 'TomoS',
 'TuteN': 'puguke',
 'Zenuk': 'budupa',
 'Zet

In [44]:
newStems=replace_keys(prevStems,racineChanges)

In [45]:
with open(repertoire+'randomStems.yaml', 'w') as output:
    yaml.dump(newStems, output, default_flow_style=False,allow_unicode=True)

In [46]:
with open(repertoire+"randomStems.yaml", 'r') as input:
    yamlLines=input.readlines()

In [47]:
yamlText="".join(yamlLines)
yamlText=re.sub(r"\n\s*-\s*",", ",yamlText)
yamlText=re.sub(r":,\s*",": ",yamlText)
yamlText=re.sub(r": *([^\n]+)",": [\g<1>]",yamlText)
print yamlText

ADJ:
  DoTaT:
    bakud: [blessé, blessés, blessée, blessées]
    biNer: [vilain, vilains, vilaine, vilaines]
    biToS: [rouge, rouges]
    davuD: [courageux, courageuse, courageuses]
    diwir: [petit, petits, petite, petites]
    jenok: [différent, différents, différente, différentes]
    liSoZ: [treize]
    loDit: [autre, autres]
    luzuN: [nouveau, nouvelle, nouvelles, nouveaux]
    magol: [quatre]
    oraNoba: [troisième, troisièmes]
    poDipo: [méchant, méchante, méchantes, méchants]
    tapuD: [jaune, jaunes]
    tarar: [vert, verte, vertes, verts]
    tifib: [grand, grands, grande, grandes]
    vulub: [disparu, disparues, disparue, disparus]
    wunot: [blanc, blancs, blanche, blanches]
  zewit:
    NuSuN: [deux]
    SonuS: [héroïque, héroïques]
    Tapat: [gros, grosse, grosses]
    ZuZog: [suivant, suivants, suivante, suivantes]
    funeT: [maigre, maigres]
    gidug: [effrayé, effrayés, effrayée, effrayées]
    gpeD: [six]
    kised: [profond, profonds, profonde, profonde

In [48]:
with open(repertoire+'randomStems.yaml', 'w') as output:
    output.write(yamlText)